# Process five-echo flashing checkerboard dataset for software demo

In [ ]:
%matplotlib inline
import os
import os.path as op
from glob import glob
import webbrowser

from tedana.workflows import tedana_workflow

## Download 5 echo data

In [ ]:
%%time
dset_dir5 = 'five-echo-dataset/'
wd = os.getcwd()

if not op.isdir(dset_dir5):
    os.mkdir(dset_dir5)

!curl -L -o five_echo_NIH.tar.xz https://osf.io/ea5v3/download
!tar xf five_echo_NIH.tar.xz -C five-echo-dataset
os.remove('five_echo_NIH.tar.xz')

## Run workflow on 5 echo data

In [ ]:
%%time
dset_dir5_out = f"{dset_dir5}tedana_processed"
files = sorted(glob(op.join(dset_dir5, 'p06*.nii.gz')))
tes = [15.4, 29.7, 44.0, 58.3, 72.6]
tedana_workflow(files, tes, 
    tree="minimal",
    fixed_seed=42,
    ica_method="robustica",
    n_robust_runs=30,
    tedpca=53,
    out_dir=dset_dir5_out,
    tedort=False
    )

## Open report for 5 echo data in web browser

In [ ]:
new = 2 # open in a new tab, if possible

url = str(os.path.abspath(dset_dir5_out + '/tedana_report.html'))
print(url)
webbrowser.open("file://" + url,new=new)


### Time series generation using AFNI commands

These commands won't run without AFNI,
but the outputted external_regressors.tsv is in this repo
along with the commands used to generate it.

With AFNI installed, this can be run from within ohbm-2025-multiecho/five-echo-dataset

```bash
# A rough CSF mask for demonstration purposes
# Segment the CSF, erode by 1 voxel, #
# and retain voxels containing 75% of the CSF mask when downsampled to EPI space
3dSeg -anat SBJ01_Anatomy.nii.gz -mask AUTO \
    -classes 'CSF ; GM ; WM'  \
    -bias_classes 'GM ; WM' \
    -bias_fwhm 25 -mixfrac UNI -main_N 5 \
    -blur_meth BFT
3dcalc -a ./Segsy/Classes+tlrc -expr 'equals(a, 1)' -prefix CSF_anatresolution.nii.gz
3dmask_tool -input CSF_anatresolution.nii.gz \
    -prefix CSF_eroded.nii.gz \
    -dilate_result -1 -fill_holes 
3dfractionize -template p06.SBJ01_S09_Task11_e3.sm.nii.gz \
    -prefix CSF_mask.nii.gz \
    -input CSF_eroded.nii.gz \
    -clip 0.75

# make CSF principal components
3dpc -mask CSF_mask.nii.gz -pcsave 3  \
    -prefix CSF_timeseries \
    ./tedana_processed/desc-optcom_bold.nii.gz

# Combine all external regressors into a single file
external_regress_header="mot_x\tmot_y\tmot_z\tmot_pitch\tmot_roll\tmot_yaw\t"\
"mot_dx\tmot_dy\tmot_dz\tmot_dpitch\tmot_droll\tmot_dyaw\tcsf1\tcsf2\tcsf3\t"\
"signal_checkerboard"
1dcat -tsvout \
    SBJ01_S09_Task11_e2_Motion.demean.1D \
    SBJ01_S09_Task11_e2_Motion.demean.der.1D \
    CSF_timeseries0?.1D \
    block_task_response.1D \
    > tmp.tsv
    (echo $external_regress_header; tail -n +2 tmp.tsv) > external_regressors.tsv 
```

## Run workflow on 5 echo data using existing mixing matrix and external regressors

In [ ]:
%%time
dset_dir5_extern_out = f"{dset_dir5}tedana_external_regress_processed"
files = sorted(glob(op.join(dset_dir5, 'p06*.nii.gz')))
tes = [15.4, 29.7, 44.0, 58.3, 72.6]
tedana_workflow(files, tes, 
    tree="demo_external_regressors_motion_task_models",
    external_regressors=op.join(dset_dir5,"external_regressors.tsv"),
    mixing_file=op.join(dset_dir5,"tedana_processed", "desc-ICA_mixing.tsv"),
    out_dir=dset_dir5_extern_out
    )

## Open report for 5 echo data with external regressors in web browser

In [ ]:
new = 2 # open in a new tab, if possible

url = str(os.path.abspath(dset_dir5_extern_out + '/tedana_report.html'))
print(url)
webbrowser.open("file://" + url,new=new)